Notebook to test random stuff

In [1]:
import pandas as pd 
import numpy as np
import torch
from torch.utils.data import Dataset
from torch import nn

from library.util import bootstrap_labeled_sets, aggregate_raw_signal
from library.datasets import Binned_Signal_Dataset

In [38]:
index = pd.MultiIndex.from_product(
    [
        ["images", "deep_sets", "event_by_event"],
        [70_000, 24_000, 6_000]
    ],
    names=["method", "num_events_per_set"]
)

t = pd.DataFrame(
    index=index, 
    columns=["MSE", "MAE", "Std. at NP", "Mean at NP", "Bias at NP"]
)

In [39]:
t

MSE  MAE Std. at NP Mean at NP Bias at NP
method         num_events_per_set                                           
images         70000               NaN  NaN        NaN        NaN        NaN
               24000               NaN  NaN        NaN        NaN        NaN
               6000                NaN  NaN        NaN        NaN        NaN
deep_sets      70000               NaN  NaN        NaN        NaN        NaN
               24000               NaN  NaN        NaN        NaN        NaN
               6000                NaN  NaN        NaN        NaN        NaN
event_by_event 70000               NaN  NaN        NaN        NaN        NaN
               24000               NaN  NaN        NaN        NaN        NaN
               6000                NaN  NaN        NaN        NaN        NaN

In [40]:
t.loc[("images", 70_000), "MSE"] = 5

In [41]:
t

MSE  MAE Std. at NP Mean at NP Bias at NP
method         num_events_per_set                                           
images         70000                 5  NaN        NaN        NaN        NaN
               24000               NaN  NaN        NaN        NaN        NaN
               6000                NaN  NaN        NaN        NaN        NaN
deep_sets      70000               NaN  NaN        NaN        NaN        NaN
               24000               NaN  NaN        NaN        NaN        NaN
               6000                NaN  NaN        NaN        NaN        NaN
event_by_event 70000               NaN  NaN        NaN        NaN        NaN
               24000               NaN  NaN        NaN        NaN        NaN
               6000                NaN  NaN        NaN        NaN        NaN

In [43]:
t = pd.DataFrame({"a":[1,2,3], "b":[1,2,3]})
t

,a,b
0,1,1
1,2,2
2,3,3


In [46]:
print(t.to_latex())

\begin{tabular}{lrr}
\toprule
 & a & b \\
\midrule
0 & 1 & 1 \\
1 & 2 & 2 \\
2 & 3 & 3 \\
\bottomrule
\end{tabular}



In [18]:
t = torch.Tensor([0.1, 0.2, 0.3])
i = torch.Tensor([0,1,1])
t[i.int()]

tensor([0.1000, 0.2000, 0.2000])

In [13]:
np.argwhere(t == 0.1)

tensor([[0]])

In [ ]:
def make_predictions(
    model, 
    features, 
    device, 
    event_by_event=False, 
    bin_values=None
):
    """
    Make predictions on an array of features.
    """
    with torch.no_grad():
        predictions = []
        for feat in features:
            if event_by_event:
                prediction = model.calculate_expected_value(
                    features.to(device),
                    bin_values.to(device),
                )
            else:
                prediction = model(feat.unsqueeze(0).to(device))
            predictions.append(prediction)
        predictions = torch.tensor(predictions)
    return predictions

def run_linearity_test(predictions, labels):
    """
    Calculate the average and standard deviation of
    predictions for each label.

    DANGER: Assumes data sorted by labels.
    """
    with torch.no_grad():
        num_sets_per_label = get_num_per_unique_label(labels)
        avg_yhat_per_label = predictions.reshape(-1, num_sets_per_label).mean(dim=1)
        std_yhat_per_label = predictions.reshape(-1, num_sets_per_label).std(dim=1)
        unique_labels = torch.unique(labels)
    return unique_labels, avg_yhat_per_label, std_yhat_per_label

def calculate_mse_mae(predictions, labels):
    """
    Calculate the mean squared error and the mean absolute error.
    """
    with torch.no_grad():
        mse = torch.nn.functional.mse_loss(predictions, labels)
        mae = torch.nn.functional.l1_loss(predictions, labels)
    return mse, mae

In [ ]:
def run_sensitivity_test(predictions, label):
    mean = predictions.mean()
    std = predictions.std()
    bias = mean - label
    return mean, std, bias